# Grafo Verb <-> Verb (Hiperlink)

In [ ]:
# CÉLULA 1: CONFIGURAÇÃO E FUNÇÕES AUXILIARES
# Execute esta célula uma vez para carregar as configurações e funções na memória.

import json
import os
import networkx as nx
import community as community_louvain
import random
from pathlib import Path
from typing import List, Dict, Any, Tuple
import math
import pickle # Usaremos pickle para salvar e carregar as distâncias de forma eficiente

# --- 1. SEÇÃO DE CONFIGURAÇÃO CENTRALIZADA ---
INPUT_DIR = Path('dados')
# Ficheiro de entrada (resultado do script de coleta)
INPUT_FILENAME = 'dados_api_20250812_125616.json' 
# Ficheiros de saída intermediários e finais
METRICS_OUTPUT_FILENAME = 'dados_com_metricas_v2.json'
POSITIONS_OUTPUT_FILENAME = 'dados_com_posicoes_v2.json'
FINAL_REFINED_OUTPUT_FILENAME = 'dados_com_posicoes_v3.json'
SHORTEST_PATHS_FILENAME = 'shortest_paths_dist.pkl' # Ficheiro para as distâncias

# Parâmetros dos algoritmos
LOUVAIN_RESOLUTION = 0.9
RANDOM_SEED = 42
LAYOUT_SCALE_FACTOR = 8000

print("✅ Célula 1: Configurações e funções carregadas.")

# --- FUNÇÕES AUXILIARES ---

def carregar_dados_json(filepath: Path) -> List[Dict[str, Any]]:
    """Carrega a lista de verbetes de um arquivo JSON."""
    print(f"Carregando dados de '{filepath}'...")
    if not filepath.exists():
        print(f"ERRO: O arquivo de entrada não foi encontrado em '{filepath}'")
        return []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            dados = json.load(f)
        return dados.get('verbetes_completo', [])
    except json.JSONDecodeError:
        print(f"ERRO: O arquivo '{filepath}' não é um JSON válido.")
        return []

def construir_grafo(verbetes: List[Dict[str, Any]], direcionado=True) -> nx.Graph:
    """Constrói um grafo a partir da lista de verbetes."""
    print(f"Construindo grafo {'direcionado' if direcionado else 'não-direcionado'}...")
    G = nx.DiGraph() if direcionado else nx.Graph()
    titulos_ids = {v['titulo']: v['id'] for v in verbetes}
    for verbete in verbetes:
        G.add_node(str(verbete['id']))
    for verbete in verbetes:
        source_vid = str(verbete['id'])
        for ref_titulo in verbete.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                if G.has_node(source_vid) and G.has_node(target_vid):
                    G.add_edge(source_vid, target_vid)
    print(f"Grafo criado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")
    return G

def salvar_dados_json(data: List[Dict[str, Any]], filepath: Path):
    """Salva a lista de verbetes em um arquivo JSON."""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump({'verbetes_completo': data}, f, ensure_ascii=False, indent=2)
    print(f"✅ Arquivo salvo com sucesso em: '{filepath}'")



✅ Célula 1: Configurações e funções carregadas.


In [15]:
# CÉLULA 2: CÁLCULO DE MÉTRICAS DE REDE
# Esta célula carrega os dados brutos, calcula todas as métricas de rede
# e salva um arquivo intermediário. Execute-a apenas quando os dados de entrada mudarem.

def calcular_e_enriquecer_metricas():
    # Executa a primeira parte do pipeline: carrega dados brutos,
    # constrói o grafo e calcula todas as métricas de rede.
    
    input_path = INPUT_DIR / INPUT_FILENAME
    metrics_output_path = INPUT_DIR / METRICS_OUTPUT_FILENAME

    verbetes = carregar_dados_json(input_path)
    if not verbetes: return

    G = construir_grafo(verbetes, direcionado=True)
    
    print("\nCalculando métricas de rede complexas...")
    G_undirected = G.to_undirected()
    
    # Detecção de comunidades no componente gigante
    print(" - Detectando comunidades no componente gigante...")
    connected_components = list(nx.connected_components(G_undirected))
    partition = {}
    if connected_components:
        giant_component_nodes = max(connected_components, key=len)
        G_giant = G_undirected.subgraph(giant_component_nodes)
        partition = community_louvain.best_partition(G_giant, resolution=LOUVAIN_RESOLUTION, random_state=RANDOM_SEED)

    num_communities = len(set(partition.values()))
    print(f"Foram detectadas {num_communities} comunidades no componente gigante.")
    
    # Cores de alto contraste
    cores_distintas = ["#e6194B", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#bcf60c", "#fabebe", "#008080", "#e6beff", "#9A6324", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000075", "#a9a9a9"]
    random.seed(RANDOM_SEED)
    community_colors = {i: cores_distintas[i % len(cores_distintas)] for i in range(num_communities)}

    # Cálculo de centralidades
    print(" - Calculando Centralidades...")
    metrics = {
        'partition': partition,
        'in_degree': dict(G.in_degree()),
        'out_degree': dict(G.out_degree()),
        'total_degree': dict(G.degree()),
        'betweenness': nx.betweenness_centrality(G),
        'pagerank': nx.pagerank(G),
        'closeness': nx.closeness_centrality(G),
        'clustering': nx.clustering(G_undirected)
    }

    # Enriquecimento dos dados
    print("Enriquecendo o dataset com as métricas calculadas...")
    for verbete in verbetes:
        vid = str(verbete['id'])
        community_id = partition.get(vid, -1)
        verbete.update({
            'community_id': community_id,
            'community_color': community_colors.get(community_id, '#6A737D'),
            'in_degree': metrics['in_degree'].get(vid, 0),
            'out_degree': metrics['out_degree'].get(vid, 0),
            'total_degree': metrics['total_degree'].get(vid, 0),
            'betweenness_centrality': metrics['betweenness'].get(vid, 0.0),
            'pagerank': metrics['pagerank'].get(vid, 0.0),
            'clustering_coefficient': metrics['clustering'].get(vid, 0.0),
            'closeness_centrality': metrics['closeness'].get(vid, 0.0)
        })
        
    salvar_dados_json(verbetes, metrics_output_path)

# Executa o cálculo de métricas
calcular_e_enriquecer_metricas()


Carregando dados de 'dadosWikifavelas20250511\dados_completos_20250812_125616.json'...
Construindo grafo direcionado...
Grafo criado com 3552 nós e 15301 arestas.

Calculando métricas de rede complexas...
 - Detectando comunidades no componente gigante...
Foram detectadas 19 comunidades no componente gigante.
 - Calculando Centralidades...
Enriquecendo o dataset com as métricas calculadas...
✅ Arquivo salvo com sucesso em: 'dadosWikifavelas20250511\dados_com_metricas_v2.json'


In [16]:
# CÉLULA 3: CÁLCULO DE LAYOUT INICIAL (KAMADA-KAWAI + GRELHA)
# Esta célula carrega os dados com métricas e calcula as posições iniciais dos nós.
# A otimização do 'dist' fará com que ela seja muito mais rápida após a primeira execução.

def calcular_e_salvar_layout_inicial():
    # Carrega os dados com métricas e calcula as posições iniciais dos nós,
    # otimizando o cálculo de distâncias.
    
    metrics_path = INPUT_DIR / METRICS_OUTPUT_FILENAME
    positions_output_path = INPUT_DIR / POSITIONS_OUTPUT_FILENAME
    shortest_paths_path = INPUT_DIR / SHORTEST_PATHS_FILENAME
    
    verbetes = carregar_dados_json(metrics_path)
    if not verbetes: return

    # Reconstrói o grafo não-direcionado para componentes e distâncias
    G_undirected = construir_grafo(verbetes, direcionado=False)

    # Separa componente gigante dos isolados
    connected_components = list(nx.connected_components(G_undirected))
    giant_component_nodes = max(connected_components, key=len)
    G_giant = G_undirected.subgraph(giant_component_nodes)
    isolated_nodes = [node for node in G_undirected.nodes() if node not in giant_component_nodes]

    # Otimização: Calcula ou carrega as distâncias do caminho mais curto
    if shortest_paths_path.exists():
        print(f"Carregando distâncias pré-calculadas de '{shortest_paths_path}'...")
        with open(shortest_paths_path, 'rb') as f:
            shortest_path_dist = pickle.load(f)
    else:
        print("Calculando distâncias de caminho mais curto (pode demorar)...")
        path_length = nx.all_pairs_shortest_path_length(G_giant)
        shortest_path_dist = {source: targets for source, targets in path_length}
        with open(shortest_paths_path, 'wb') as f:
            pickle.dump(shortest_path_dist, f)
        print(f"Distâncias salvas em '{shortest_paths_path}' para uso futuro.")

    # Calcula layout do componente gigante usando as distâncias pré-calculadas
    print(f" - Calculando Kamada-Kawai para {G_giant.number_of_nodes()} nós...")
    posicoes_relativas = nx.kamada_kawai_layout(G_giant, dist=shortest_path_dist, scale=LAYOUT_SCALE_FACTOR)
    
    posicoes_finais = {node_id: {'x': pos[0], 'y': pos[1]} for node_id, pos in posicoes_relativas.items()}
    
    # Posiciona os nós isolados em grelha
    print(f" - Posicionando {len(isolated_nodes)} nós isolados em grelha...")
    if isolated_nodes:
        max_x_main_graph = max(pos['x'] for pos in posicoes_finais.values()) if posicoes_finais else 0
        grid_start_x = max_x_main_graph + (LAYOUT_SCALE_FACTOR * 0.3)
        spacing = 300
        cols = int(math.sqrt(len(isolated_nodes))) + 1
        for i, node_id in enumerate(isolated_nodes):
            row = i // cols
            col = i % cols
            posicoes_finais[node_id] = {'x': grid_start_x + (col * spacing), 'y': row * spacing}

    # Adiciona posições ao dataset
    for verbete in verbetes:
        verbete['position'] = posicoes_finais.get(str(verbete['id']), {'x': 0, 'y': 0})
        
    salvar_dados_json(verbetes, positions_output_path)

# Executa o cálculo de layout inicial
calcular_e_salvar_layout_inicial()


Carregando dados de 'dadosWikifavelas20250511\dados_com_metricas_v2.json'...
Construindo grafo não-direcionado...
Grafo criado com 3552 nós e 14128 arestas.
Carregando distâncias pré-calculadas de 'dadosWikifavelas20250511\shortest_paths_dist.pkl'...
 - Calculando Kamada-Kawai para 3337 nós...
 - Posicionando 215 nós isolados em grelha...
✅ Arquivo salvo com sucesso em: 'dadosWikifavelas20250511\dados_com_posicoes_v2.json'


In [17]:
# CÉLULA 4: REFINAMENTO DO LAYOUT PARA EVITAR SOBREPOSIÇÃO
# Esta célula é ideal para ser executada várias vezes, ajustando os parâmetros
# do spring_layout para encontrar a melhor visualização.

def refinar_e_salvar_layout_final():
    #Carrega os dados com posições iniciais e aplica o spring_layout
    # para refinar o posicionamento e evitar sobreposição.
    
    positions_path = INPUT_DIR / POSITIONS_OUTPUT_FILENAME
    final_output_path = INPUT_DIR / FINAL_REFINED_OUTPUT_FILENAME
    
    verbetes = carregar_dados_json(positions_path)
    if not verbetes: return

    G_undirected = construir_grafo(verbetes, direcionado=False)

    print("\nIniciando o refinamento do layout para evitar sobreposição...")
    posicoes_iniciais_scaled = {str(v['id']): (v['position']['x'], v['position']['y']) for v in verbetes if 'position' in v}

    # Normaliza as posições para a escala ideal do spring_layout [-1, 1]
    max_abs_coord = max(abs(coord) for pos in posicoes_iniciais_scaled.values() for coord in pos)
    if max_abs_coord == 0: max_abs_coord = 1.0
    posicoes_iniciais_unscaled = {node: (x / max_abs_coord, y / max_abs_coord) for node, (x, y) in posicoes_iniciais_scaled.items()}

    # Identifica o componente gigante novamente
    connected_components = list(nx.connected_components(G_undirected))
    giant_component_nodes = max(connected_components, key=len)
    G_giant = G_undirected.subgraph(giant_component_nodes)
    
    pos_giant_iniciais_unscaled = {node: pos for node, pos in posicoes_iniciais_unscaled.items() if node in giant_component_nodes}

    print(f" - Refinando {len(pos_giant_iniciais_unscaled)} nós do componente gigante...")
    pos_ajustadas_giant_unscaled = nx.spring_layout(
        G_giant, 
        pos=pos_giant_iniciais_unscaled, 
        iterations=80,
        seed=RANDOM_SEED,
        k=0.4 / math.sqrt(G_giant.number_of_nodes()) # Parâmetro 'k' para ajustar o espaçamento
    )

    # Combina e re-escala as posições
    posicoes_finais_refinadas = {}
    for node_id, pos_tuple_scaled in posicoes_iniciais_scaled.items():
        if node_id in pos_ajustadas_giant_unscaled:
            final_pos_unscaled = pos_ajustadas_giant_unscaled[node_id]
            posicoes_finais_refinadas[node_id] = {'x': float(final_pos_unscaled[0] * max_abs_coord), 'y': float(final_pos_unscaled[1] * max_abs_coord)}
        else: # Mantém a posição original para os nós isolados
            posicoes_finais_refinadas[node_id] = {'x': float(pos_tuple_scaled[0]), 'y': float(pos_tuple_scaled[1])}
            
    # Atualiza o dataset final
    for verbete in verbetes:
        verbete_id = str(verbete['id'])
        if verbete_id in posicoes_finais_refinadas:
            verbete['position'] = posicoes_finais_refinadas[verbete_id]
    
    salvar_dados_json(verbetes, final_output_path)

# Executa o refinamento final
refinar_e_salvar_layout_final()


Carregando dados de 'dadosWikifavelas20250511\dados_com_posicoes_v2.json'...
Construindo grafo não-direcionado...
Grafo criado com 3552 nós e 14128 arestas.

Iniciando o refinamento do layout para evitar sobreposição...
 - Refinando 3337 nós do componente gigante...
✅ Arquivo salvo com sucesso em: 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'


In [ ]:
# calcular_metrica_composta.py
#
# Este script lê os dados finais da rede e calcula uma métrica composta
# para identificar os "super-nós" — verbetes que são importantes em múltiplas dimensões.
#
# Metodologia Estatística:
# 1. Normalização Min-Max: Cada uma das cinco métricas selecionadas é normalizada
#    para uma escala de 0 a 1. Isso garante que cada métrica contribua de forma
#    equitativa para o resultado final, independentemente da sua escala original.
#    Fórmula: valor_normalizado = (valor - min) / (max - min)
# 2. Soma Ponderada (com pesos iguais): A métrica composta é a soma das
#    métricas normalizadas. Esta abordagem é mais robusta que a multiplicação,
#    pois não anula a pontuação de um nó que seja fraco em uma única dimensão.

import json
from pathlib import Path
import pandas as pd

def calcular_metrica_composta():
    """
    Calcula uma métrica de importância composta normalizando e somando
    diferentes indicadores de centralidade e atividade.
    """
    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dadosWikifavelas20250511')
    INPUT_FILENAME = 'dados_com_posicoes_v3.json'
    OUTPUT_FILENAME = 'dados_com_metrica_composta.json'
    
    input_path = INPUT_DIR / INPUT_FILENAME
    output_path = INPUT_DIR / OUTPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de entrada não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS ---
    print(f"Carregando dados de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes = dados.get('verbetes_completo', [])
    if not verbetes:
        print("Nenhum verbete encontrado no arquivo.")
        return
        
    # Usar o Pandas facilita muito a normalização de colunas
    df = pd.DataFrame(verbetes)
    
    # Lista das métricas que farão parte do índice composto
    metricas_para_normalizar = [
        'betweenness_centrality',
        'closeness_centrality',
        'total_degree',
        'quantidade_edicoes',
        'pagerank'
    ]

    print("\nNormalizando as métricas para uma escala de 0 a 1...")
    for metrica in metricas_para_normalizar:
        if metrica in df.columns:
            min_val = df[metrica].min()
            max_val = df[metrica].max()
            
            # Evita divisão por zero se todos os valores forem iguais
            if (max_val - min_val) > 0:
                df[f'{metrica}_norm'] = (df[metrica] - min_val) / (max_val - min_val)
            else:
                df[f'{metrica}_norm'] = 0 # Se não há variação, a contribuição é nula
            
            print(f" - Métrica '{metrica}' normalizada.")
        else:
            print(f" - AVISO: Métrica '{metrica}' não encontrada no dataset.")
            df[f'{metrica}_norm'] = 0 # Adiciona uma coluna de zeros se a métrica não existir

    # --- 3. CÁLCULO DA MÉTRICA COMPOSTA ---
    print("\nCalculando a métrica composta pela soma dos valores normalizados...")
    
    # A métrica composta é a soma das versões normalizadas de cada indicador
    df['metrica_composta'] = (
        df['betweenness_centrality_norm'] +
        df['closeness_centrality_norm'] +
        df['total_degree_norm'] +
        df['quantidade_edicoes_norm'] +
        df['pagerank_norm']
    )
    
    print("Métrica composta calculada com sucesso.")

    # --- 4. ANÁLISE DOS RESULTADOS ---
    print("\n--- TOP 20 VERBETES POR MÉTRICA COMPOSTA ---")
    
    # Ordena o DataFrame pela nova métrica para encontrar os nós mais importantes
    df_sorted = df.sort_values(by='metrica_composta', ascending=False)
    
    for index, row in df_sorted.head(20).iterrows():
        print(f"  - Título: {row['titulo']:<70} | Pontuação Composta: {row['metrica_composta']:.4f}")

    # --- 5. SALVANDO O ARQUIVO FINAL ---
    # Converte o DataFrame de volta para o formato de dicionário original
    verbetes_finais = df_sorted.to_dict(orient='records')
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({'verbetes_completo': verbetes_finais}, f, ensure_ascii=False, indent=2)
        
    print(f"\n✅ Arquivo com a métrica composta salvo em '{output_path}'!")


if __name__ == '__main__':
    calcular_metrica_composta()


In [ ]:
# calcular_constraint.py
#
# Este script lê os dados enriquecidos da rede, reconstrói o grafo
# e calcula a métrica de Restrição (Constraint) de Burt para cada nó.
# A métrica é então adicionada ao dataset, que é salvo em um novo arquivo.

import json
from pathlib import Path
import networkx as nx
from typing import List, Dict, Any

def calcular_constraint_para_nos():
    """
    Calcula a métrica de Restrição (Constraint) e a adiciona ao
    conjunto de dados de verbetes.
    """
    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dadosWikifavelas20250511')
    
    # Ficheiro de entrada que já contém todas as outras métricas
    INPUT_FILENAME = 'dados_com_metrica_composta.json'
    
    # Novo ficheiro de saída que incluirá a métrica de constraint
    OUTPUT_FILENAME = 'dados_com_constraint.json'
    
    input_path = INPUT_DIR / INPUT_FILENAME
    output_path = INPUT_DIR / OUTPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de entrada não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ---
    print(f"Carregando dados de '{input_path}'...")
    try:
        with open(input_path, 'r', encoding='utf-8') as f:
            dados = json.load(f)
    except json.JSONDecodeError:
        print(f"ERRO: O arquivo '{input_path}' não é um JSON válido.")
        return
    
    verbetes = dados.get('verbetes_completo', [])
    if not verbetes:
        print("Nenhum verbete encontrado no arquivo.")
        return
        
    # --- 3. RECONSTRUÇÃO DO GRAFO ---
    print("Reconstruindo o grafo a partir dos dados...")
    # Usamos um DiGraph (direcionado) pois a constraint em NetworkX considera
    # as ligações de saída.
    G = nx.DiGraph()
    titulos_ids = {v['titulo']: v['id'] for v in verbetes}
    
    # Adiciona todos os nós
    for verbete in verbetes:
        G.add_node(str(verbete['id']))
        
    # Adiciona todas as arestas
    for verbete in verbetes:
        source_vid = str(verbete['id'])
        for ref_titulo in verbete.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                if G.has_node(source_vid) and G.has_node(target_vid):
                    G.add_edge(source_vid, target_vid)
    
    print(f"Grafo reconstruído com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")

    # --- 4. CÁLCULO DA MÉTRICA DE CONSTRAINT ---
    print("\nCalculando a métrica de Restrição (Constraint) para cada nó...")
    # O cálculo pode levar alguns momentos dependendo do tamanho do grafo
    constraint_scores = nx.constraint(G)
    print("Cálculo da métrica de Constraint concluído.")

    # --- 5. ENRIQUECIMENTO DOS DADOS ---
    print("Adicionando a nova métrica ao conjunto de dados...")
    for verbete in verbetes:
        verbete_id = str(verbete['id'])
        # Adiciona o valor da constraint ao dicionário de cada verbete
        # Um valor BAIXO de constraint é o que indica um "nó-ponte"
        verbete['constraint'] = constraint_scores.get(verbete_id, None)

    # --- 6. SALVANDO O ARQUIVO FINAL ---
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({'verbetes_completo': verbetes}, f, ensure_ascii=False, indent=2)
        
    print(f"\n✅ Arquivo com a métrica de Constraint salvo em '{output_path}'!")
    print("Lembre-se: nós com BAIXO valor de 'constraint' são os mais interessantes para análise de 'buracos estruturais'.")

if __name__ == '__main__':
    calcular_constraint_para_nos()


Carregando dados de 'dadosWikifavelas20250511\dados_com_metrica_composta.json'...
Reconstruindo o grafo a partir dos dados...
Grafo reconstruído com 3552 nós e 15301 arestas.

Calculando a métrica de Restrição (Constraint) para cada nó...
Cálculo da métrica de Constraint concluído.
Adicionando a nova métrica ao conjunto de dados...

✅ Arquivo com a métrica de Constraint salvo em 'dadosWikifavelas20250511\dados_com_constraint.json'!
Lembre-se: nós com BAIXO valor de 'constraint' são os mais interessantes para análise de 'buracos estruturais'.
